***
## 3.DATA PREPARATION

We will clean, and transform diverse data sources, including restaurant attributes, cuisine, and historical interactions. Ensuring data quality and consistency is essential for accurate model training. The prepared dataset forms the foundation for building a robust and personalized recommendation engine."

***

In [1]:
# importing libraries
import re
import pickle
import warnings 
import numpy as np
import pandas as pd
import os
warnings.filterwarnings('ignore')
from classes.understanding import DataLoader, DataChecks



#### i) User Review Data
***


In [2]:
# Instantiate the DataLoader class
loader= DataLoader()


# Loading the users csv file
users_data= loader.read_data("data/users.csv")



In [3]:
# Checking the duplicates in the user data
checks= DataChecks(users_data)
checks.check_duplicates()

,user_id,business_id,stars,date
988470,wAph_egpAXr9ic-A2vnINQ,fiGh8ctcvIRm4Ii2GOtoHQ,2,2018-07-23 21:43:33
997259,wAph_egpAXr9ic-A2vnINQ,fiGh8ctcvIRm4Ii2GOtoHQ,2,2018-07-23 21:43:33
1980862,xZ0ioQJ0Jyo_Uc0FzmyFQQ,O_kAd89gPls-dXda40NWzA,5,2019-11-22 07:18:46
2005725,xZ0ioQJ0Jyo_Uc0FzmyFQQ,O_kAd89gPls-dXda40NWzA,5,2019-11-22 07:18:46


In [4]:
# Dropping the duplicates
users_data_cleaned= users_data.drop_duplicates()

# Confirming the duplicates have been dropped
checks= DataChecks(users_data_cleaned)
checks.check_duplicates()

NO DUPLICATES FOUND


In [5]:
#Checking the first five rows fot the user_data
users_data_cleaned.head()

,user_id,business_id,stars,date
0,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,2018-07-07 22:09:11
1,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,2017-01-14 20:54:15
2,RreNy--tOmXMl1en0wiBOg,cPepkJeRMtHapc_b2Oe_dw,4,2018-07-17 03:30:07
3,Jha0USGDMefGFRLik_xFQg,bMratNjTG5ZFEA6hVyr-xQ,5,2017-02-19 13:32:05
4,4hBhtCSgoxkrFgHa4YAD-w,bbEXAEFr4RYHLlZ-HFssTA,5,2017-01-02 03:17:34


In [6]:
users_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2559584 entries, 0 to 2559585
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      object
 1   business_id  object
 2   stars        int64 
 3   date         object
dtypes: int64(1), object(3)
memory usage: 97.6+ MB


 **Observations**
 ***
 
> The duplicated rows were dropped in preparation for modelling.

> The data has 2,559,584 rows and 4 columns.

#### ii) Restaurant Informational Data
***

In [7]:
# Reading the restaurants csv file
restaurant_data= loader.read_data("data/restaurants.csv")

# Looking into the first five rows of the data frame
restaurant_data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."


In [8]:
# Checking for null values in the dataset
restaurant_data.isna().sum()

business_id        0
name               0
address          443
city               0
state              0
postal_code       21
latitude           0
longitude          0
stars              0
review_count       0
is_open            0
attributes       566
categories         0
hours           7279
dtype: int64

**Observation**
***

The dataset contains null values in the columns: address, postal code, attributes and hours

In [9]:
# Dropping null values
restaurant_data = restaurant_data.dropna(subset=['attributes'])

In [10]:
# Filling missing values in the 'attributes' column with 'Unknown'
restaurant_data[['address','postal_code','hours']]=restaurant_data[['address','postal_code','hours']].fillna('Unknown')

# Verifying that all missing values have been handled
restaurant_data.isnull().sum()


business_id     0
name            0
address         0
city            0
state           0
postal_code     0
latitude        0
longitude       0
stars           0
review_count    0
is_open         0
attributes      0
categories      0
hours           0
dtype: int64

**Observations**
***
 There were some missing values in the `attributes`, `address`, `postal_code`, and `hours` columns.
 
 We handled the missing data in two ways:
 
 1) **Dropping the records with null values**

- Rows with missing `attributes` were removed as this column is critical for analysis.

2) **Imputation**

- Missing values in `address`, `postal_code`, and `hours` were imputed  with 'Unknown' to maintain data integrity.

In [11]:
# Checking the value counts for the states
restaurant_data.state.value_counts()

state
PA     12538
FL      8632
TN      4297
MO      4219
IN      4119
LA      3596
NJ      3302
AZ      2635
AB      2355
NV      1657
ID      1290
CA      1151
IL       975
DE       949
NC         1
CO         1
HI         1
MT         1
XMS        1
Name: count, dtype: int64

In [12]:
# States labeled `XMS` will be filtered out, as they do not seem valid
valid_states = ['XMS','MT','NC','HI','CO',]
restaurant_data_filtered = restaurant_data[~restaurant_data['state'].isin(valid_states)]


In [13]:
# Map state abbreviations to full state names
state_abbreviations = {
    'AB': 'Alberta', 'AZ': 'Arizona', 'CA': 'California', 'DE': 'Delaware', 'FL': 'Florida', 'ID': 'Idaho','IL': 'Illinois',
    'IN': 'Indiana','LA': 'Louisiana',  'MO': 'Missouri','NV': 'Nevada', 'NJ': 'New Jersey','PA': 'Pennsylvania',  'TN': 'Tennessee', 
}
# Replace state initials with full names
restaurant_data_filtered['state'] = restaurant_data_filtered['state'].map(state_abbreviations)



**Observation**
***

- States labeled XMS,MT,NC,HI and Co were noted in the EDA for having unusually small restaurant counts, upon confirmation, they have been filtered  out as they contain only a value of one restaurant per state therefore not statistically relevant to our project.

- The state abbreviations have been successfully mapped to their corresponding full state names, providing more clarity,simplicity,coherence and consistency in the data.

### FEATURE ENGINEERING
***

We will continue with feature engineering with the following steps
1. Exploding the categories to refine out cuisine list.
2. Elimiating spaces that arise from the splits.
3. Engineering a Location column that is a combination of various features.
4. Filtering the attributes column to keep only attributes that are true.
 

In [14]:
# Exploding the categories column to ensure each category is handled separately
df_exploded = restaurant_data_filtered.assign(categories=restaurant_data_filtered['categories'].str.split(',')).explode('categories')

df_exploded.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",Restaurants,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",Food,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",Bubble Tea,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",Coffee & Tea,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,Pennsylvania,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...",Bakeries,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


In [15]:
# Function to standardize spacing
def standardize_spacing(x):
    if isinstance(x, str):
        return re.sub(r'\s+', ' ', x.strip())
    return x

# Apply the function to all elements in the DataFrame
df_exploded = df_exploded.applymap(standardize_spacing)

In [16]:
# List of all the Cuisine in the categories
cuisine=["American(New)", "Mexican",'American (Traditional)','Italian','Chinese',
         'Japanese','Asian Fusion','Mediterranean','Southern','Cajun/Creole','Tex-Mex',
         'Thai','Latin American','Indian','Vietnamese','Greek','Caribbean','Middle Eastern',
         'French','Korean','Halal','Spanish','Canadian (New)','Irish','Pakistani','Hawaiian',
         'Soul Food','German','Szechuan','African','Filipino','Lebanese','Puerto Rican','Turkish',
         'Cantonese','British','Peruvian','Kosher','Brazilian','Pan Asian','Taiwanese','Cuban','Colombian',
         'Ethiopian','Venezuelan','Salvadoran','Laotian','Polish','Dominican','Russian','Persian/Iranian',
         'Afghan','Moroccan','Arabic','Portuguese','Mongolian','Argentine','Malaysian','Belgian',
         'Honduran','Himalayan/Nepalese','Armenian','Trinidadian','Ukrainian','Australian','Egyptian']

# Filtering to get restaurants with cuisine
restaurant_exploded = df_exploded[df_exploded.categories.isin(cuisine)]

In [17]:
# Display the 'location' column to check the combined addresses
restaurant_exploded.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,Missouri,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...",Italian,Unknown
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,Missouri,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...",American (Traditional),Unknown
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,Missouri,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...",Greek,Unknown
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,Unknown,Tampa Bay,Florida,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Vietnamese,"{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
5,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,Indiana,46227,39.637133,-86.127217,2.5,28,1,"{'RestaurantsReservations': 'False', 'Restaura...",American (Traditional),"{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ..."


In [18]:
# combining the address columns into a location column
restaurant_exploded['location']=restaurant_exploded[['city','state','address']]\
            .apply( lambda x: f"State:{x['state']}, City:{x['city']}, Address:{x['address']} ", axis=1)


# viewing the restaurant_exploded dataset information
restaurant_exploded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38528 entries, 2 to 52285
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   38528 non-null  object 
 1   name          38528 non-null  object 
 2   address       38528 non-null  object 
 3   city          38528 non-null  object 
 4   state         38528 non-null  object 
 5   postal_code   38528 non-null  object 
 6   latitude      38528 non-null  float64
 7   longitude     38528 non-null  float64
 8   stars         38528 non-null  float64
 9   review_count  38528 non-null  int64  
 10  is_open       38528 non-null  int64  
 11  attributes    38528 non-null  object 
 12  categories    38528 non-null  object 
 13  hours         38528 non-null  object 
 14  location      38528 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 4.7+ MB


**Observations**
***
 
- The `categories` column have been standardized and exploded to handle multiple categories per restaurant. This transformation is necessary to accurately categorize each restaurant's offerings, which is essential for matching user preferences with the right restaurants.

- The exploded DataFrame has been filtered to include only rows with specified cuisines. By narrowing down to specific cuisines, we ensure that the recommender system focuses on the most relevant restaurant types. This helps in improving precision

- Combining the `state`, `city`, and `address` columns is vital for geographic-based recommendations, enabling the system to suggest restaurants within a user’s preferred or current location.

- This results in a dataset with 38,528 rows of restaurants with their cuisine types and 15 columns

In [19]:
def decompress(x):
    """
    The function takes in a dictionary and returns only the keys that have their values not being False   
    """
      
    list_ = []
    
    # Check if x is a string
    if not isinstance(x, str):
        return ' '
    
    # evaluate the attributes column to convert it from a string to a dictionary
    try:
        data_dict = eval(x)
    except Exception as e:
        print(f"Error evaluating {x}: {str(e)}")
        return ' '
    
    # iterate through the key-value pairs in the dictionary
    for key, val in data_dict.items():
        # check if the key is in the specified categories and if the value is not "None"
        if (key in ['Ambience', 'GoodForMeal', 'BusinessParking']) and (val != "None"):
            # if conditions are met, further iterate through sub-dictionary
            try:
                sub_dict = eval(val)
                for key_, val_ in sub_dict.items():
                    # if the sub-dictionary value is true, append it to the list
                    if val_:
                        list_.append(f'{key}{key}')
            except Exception as e:
                print(f"Error evaluating {val}: {str(e)}")
        else:
            # if the value is not false, append the key to the list
            if val != 'False':
                list_.append(key)
    
    # join the list of selected attribute names into a space-separated string
    return " ".join(list_)

# create a new column 'attributes_true' in the df by applying the decompress function
# include a condition to handle cases where attributes is 'Not-Available'
restaurant_exploded['attributes_true'] = restaurant_exploded.attributes.apply(lambda x: decompress(x) if x != 'Not-Available' else ' ')

In [20]:
# Example to show the new column generated using a sample business_id
# Temporarily set display options
pd.set_option('display.max_colwidth', None)  

# Display the attributes sections
print("Original Attributes Column",restaurant_data[restaurant_data.business_id=='k0hlBqXX-Bt0vf1op7Jr1w'].attributes[:1])
print("Attributes_true Column",restaurant_exploded[restaurant_exploded.business_id=='k0hlBqXX-Bt0vf1op7Jr1w'].attributes_true[:1])

# Reset display options to default after viewing
pd.reset_option('display.max_colwidth')


Original Attributes Column 2    {'Caters': 'True', 'Alcohol': "u'full_bar'", 'RestaurantsAttire': "u'casual'", 'RestaurantsDelivery': 'False', 'RestaurantsTakeOut': 'True', 'HasTV': 'True', 'NoiseLevel': "u'average'", 'BusinessAcceptsCreditCards': 'True', 'OutdoorSeating': 'True', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'Ambience': "{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': False}", 'RestaurantsPriceRange2': '1', 'GoodForKids': 'True', 'WiFi': "u'free'", 'RestaurantsReservations': 'False', 'RestaurantsGoodForGroups': 'True'}
Name: attributes, dtype: object
Attributes_true Column 2    Caters Alcohol RestaurantsAttire RestaurantsTakeOut HasTV NoiseLevel BusinessAcceptsCreditCards OutdoorSeating BusinessParkingBusinessParking RestaurantsPriceRange2 GoodForKids WiFi RestaurantsGoodForGroups
Name: attributes_true, d

In [23]:
restaurant_exploded.shape

(38528, 16)

**Observations:**
***
- By using the compress function, the attributes column was stripped of any attributes that were False and a new column attributes true was created that only had the true attributes of each restaurant

- This results in a dataset with 38,528 rows of restaurants with their cuisine types and 16 columns

In [21]:
# Save the cleaned user review data to a new CSV file
users_data_cleaned.to_csv('data/cleaned_users_data.csv', index=False)

In [22]:

# Save the filtered and transformed restaurant data to a new CSV file
with open('Pickled_files/restaurants_data.pkl','wb')as file:
    pickle.dump(restaurant_exploded, file)
